In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import h5py
import gdown
from collections import defaultdict
import matplotlib.pyplot as plt
####
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import models 
from tensorflow.keras import layers 
from tensorflow.keras import initializers
from tensorflow.keras.models import load_model
#
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

# 
from collections import defaultdict

In [2]:
def reconstr_loss(original_spectra, reconstructed_spectra, latent_dim):
    '''Function to calculate reconstruction loss.

    Inputs:
    - original_spectra (np.array): original spectra.
    - reconstructed_spectra (np.array): reconstruction of the original spectra from the latent representation.
    - latent_dim (integer): size of the latent space.

    Returns:
    - reconstruction loss with added penalty for the latent space size
    '''

    penalty = 0.00003
    penalty2 = 5*0.00003


    mse_loss = mean_squared_error(original_spectra, reconstructed_spectra, squared=True)
    loss_penalized = mse_loss + latent_dim*penalty + penalty2*(latent_dim > 6)

    return(loss_penalized)

In [7]:
import os
os.chdir('/blue/pdixit/hodaakl/Hackathon2021')

In [8]:
data_train = h5py.File( 'train_set.hdf5', 'r')
x_train = np.array( data_train['spectra'] )

data_val = h5py.File( 'val_set.hdf5', 'r')
x_val = np.array( data_val['spectra'] )

data_test = h5py.File('test_set.hdf5', 'r')
x_test = np.array( data_test['spectra'] )

In [9]:
## preprocess the data 
### denoise the data 
x_train_den = x_train.copy()
x_val_den = x_val.copy()
x_test_den = x_test.copy()

xidx, yidx = np.where(x_train>.9)
x_train_den[xidx, yidx] = np.ones(len(xidx))
xidx, yidx = np.where(x_val>.9)
x_val_den[xidx, yidx] = np.ones(len(xidx))
xidx, yidx = np.where(x_test>.9)
x_test_den[xidx, yidx] = np.ones(len(xidx))

from sklearn.preprocessing import StandardScaler


Scaler = StandardScaler()

x_at_n11_train = Scaler.fit_transform(x_train_den)
x_at_n11_val = Scaler.transform(x_val_den)
x_at_n11_test = Scaler.transform(x_test_den)

In [10]:
mod = load_model('StandardScalerModel.h5')

2021-11-30 16:54:13.126645: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-30 16:54:13.126702: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-30 16:54:13.126740: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0706a-s17.ufhpc): /proc/driver/nvidia/version does not exist
2021-11-30 16:54:13.127089: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
mod.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_6 (Sequential)    (None, 6)                 10588094  
_________________________________________________________________
sequential_7 (Sequential)    (None, 39974)             10628062  
Total params: 21,216,156
Trainable params: 21,216,156
Non-trainable params: 0
_________________________________________________________________


In [12]:
encoder = mod.get_layer(index=0)
decoder = mod.get_layer(index=1)

In [13]:
encoder.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 264)               10553400  
_________________________________________________________________
dense_13 (Dense)             (None, 128)               33920     
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 774       
Total params: 10,588,094
Trainable params: 10,588,094
Non-trainable params: 0
_________________________________________________________________


In [14]:
decoder.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               896       
_________________________________________________________________
dense_16 (Dense)             (None, 264)               34056     
_________________________________________________________________
dense_17 (Dense)             (None, 39974)             10593110  
Total params: 10,628,062
Trainable params: 10,628,062
Non-trainable params: 0
_________________________________________________________________


In [18]:
test_set_latent_encoding = encoder(x_at_n11_test)
test_set_reconstructions = decoder(test_set_latent_encoding)
    # 
from pickle import load
Scaler2 = load(open('StandardScaler.pkl', 'rb'))
x_test_reconstructed = Scaler2.inverse_transform(test_set_reconstructions)
# x_test_reconstructed = np.reshape(x_test_reconstructed, (x_test_reconstructed.shape[0], x_test_reconstructed.shape[1] ))

rl = reconstr_loss(x_test, x_test_reconstructed, latent_dim = test_set_latent_encoding.shape[1])
print(rl)

0.0022086372084942126
